## Лабораторная работа 5

### 1.Постановка задачи (вариант 23):

Реализовать 5  процессов, в каждом по три нити c фильтром  (II ) и  общим процесс-безопасным журналом, l (Log)  c уровнями  Info, Debug, прерываниями (лекции 3-4). Работа с журналом осуществляется в фоновом режиме. 
2. Сообщение в журнале формата 
     * начало/окончание, номер- нити/процесса  
     * главный процесс, окончание    

Фильтр: разработать фильтр, выводящий слова из строки в столбик.

## Решение

# Процесс с множественными нитями  1:M
<img src="image/3_02.png" alt="Drawing" style="width: 250px;"/>

Запуск нитей в дочернем процессе
stream  - поток 
thread  - поток
flow    - поток    

##### Для процесса 1 c нитями M (1:M) определим функцию, которая запускает столько потоков, сколько требуется для выполнения пользовательской функции.

### N нитей M процессов, N:M  = 3:5
<img src="image/3_03.png" alt="Drawing" style="width: 250px;"/>


1. запускаем пять дочерних процессов.
2.  Главный (__main__) процесс блокируется пока  не закончатся все дочерние процессы.
3. Каждый дочерний процесс затем настраивает и запускает три рабочих нити. Дочерние процессы затем блокируются до завершения их рабочих нитей.
4. Каждая рабочая нить блокируется  случайным интервал в три секунды, затем выводит сообщение.
5. Когда нити процесса завершаются, и дочерний процесс продолжается и выдает сообщение.
6. Процессы полностью завершаются, а главный процесс продолжается и выдает окончательное сообщение.


## Нити и Процессы - демоны (фоновые)  

#### Идея фоновой нити и процесса  аналог "демонов" или духа (из древнегреческого), который выолняет задачи на заднем плане. Можно также называть нити и процессы демоны  демоническими.
#### Нить демон и процесс демон выполняются в фоновом режиме.

#### Нить демон и  процесс демон полезны для выполнения задач в фоновом режиме для поддержки нитей и процессов, не являющихся демонами.

Свойство "демон" может быть задано только в новом потоке, если новый поток не запущен. То есть, перед вызовом метода start()

Экземпляр новой нити наследует значение свойства "демон"  от нити родителя.

Выполнение примера сначала создается  новая нить со значением по умолчанию свойства "демон". Оно унаследуется от родительской нити, которая в этом случае является главной, а не демонической.

# Процессы демоны
##  Процесс демон является расширением нити  демон для  процесса.

#### Процесс демона можно создать в Python с помощью аргумента "демон" конструктора  multiprocessing.Process или через свойство "демон" экземпляра multiprocessing.Process.Процессы

### В примере сначала создается  новый процесс со значением по умолчанию свойства "демон". Оно наследуется от родительского процесса

#  Ведение журнала событий (log или логирование)
#### мы можем вести журнальный файл, вызвав функцию logging.basicConfig() и указывая имя файла и путь к журналу (например, application.log), уровень регистрации в файл (например, от журнала.DEBUG до журнала.CRITICAL).


#### События регистрируются с помощью вызовов функций в зависимости от типа выполняемых событий, например для отладки сообщений logging.debug().

#### мы можем добавлять информационные сообщения в код приложения, вызывая журнал.info() и передавая в строке подробности события.

#### Log oшибки, такие как исключения, которые важны, но не критичны для программы через logging.error() или logging.exception()

# Log для нескольких процессов 
####  1. Используйте модуль регистрации отдельно в каждом процессе
####  2. Использование пользователем процесса безопасной регистрации


1. Главный (main) процесс блокирован  до завершения дочерних процессов.
2. Каждый дочерний процесс настраивает свой собственный журнал (Log), используя обработчик потока по умолчанию, который отправляет сообщения в stderr, и устанавливает уровень журнала для отладки (debug).
3. Дочерние процессы затем выполняют свои  задачи и логируют  сообщения (посылают в журнал).
4. Дочерние процессы заканчиваются, затем главный процесс продолжается и сообщает окончательное сообщение. 

Это указывает на то, что модуль журнала может использоваться непосредственно из нескольких процессов, хотя он требует, чтобы каждый процесс настраивал инфраструктуру ведения журнала отдельно.

Опасность этого подхода заключается в том, что конечный объект, в котором хранятся лог-сообщения, не может надежно обрабатывать сообщения от нескольких процессов параллельно, в этом случае сообщения могут перезаписывать друг друга и быть потеряны или повреждены. Это особенно верно, если все регистраторы настроены на запись в один файл.

### в предыдущем примере,  ведение журнала не является  процесс-безопасным  и сообщения журнала могут быть потеряны или повреждены, если они записаны в тот же целевой поток или обработчик.

### Изменим способ получения  журнала, чтобы настроить его. Вместо вызова logger.getLogger(), вызовем multiprocessing.get_logger() Добавим StreamHandler, чтобы сообщения записывались в журнал stderr.

#### Опасность такого подхода, такая же, что и у предыдущего

###  Используйте QueHandler и процесс регистрации (Logging)
#### Это может быть достигнуто путем настройки инфраструктуры ведения журнала в каждом процессе для использования logging.handlers.QueHandler, который будет отправлять сообщения журнала в разделяемую многопроцессорную  oчередь multiprocessing.Queue. Новый  процесс затем будет потреблять сообщения из очереди один-во-время и хранить их, в этом случае на stderr с помощью logging.StreamHandlerала.StreamHandler, но это может быть так же и файл.




In [ ]:
import random
from random import randrange
from time import sleep
from threading import current_thread
from multiprocessing import current_process
from threading import Thread
from multiprocessing import Process
from multiprocessing import Queue
from logging.handlers import QueueHandler
import logging
import string

def randomword(length):
   letters = string.ascii_lowercase
   return ''.join(random.choice(letters) for i in range(length))

def logger_process(queue):
    logger = logging.getLogger('app')
    logger.addHandler(logging.StreamHandler())
    logger.setLevel(logging.DEBUG)
    while True:
        message = queue.get()
        if message is None:
            break
        logger.handle(message)

def thread_task(str,results,index):
    sleep(random.random())
    strbuff=''
    strspl=str.split()
    
    strbuff = str.replace(' ', '\n')
    strbuff = '\n' + strbuff
    thread_name = current_thread().name
    process_name = current_process().name
    results[index] = f'\n!!! Thread {thread_name} in process {process_name} done with result: {strbuff} (started with string {str})'

def process_task(queue):
    results=[None]*3
    threads=[None]*3
    logger = logging.getLogger('app')
    logger.addHandler(QueueHandler(queue))
    logger.setLevel(logging.DEBUG)
    
    for n in range(len(threads)):
        stringa=randomword(randrange(1, 5))+' '+randomword(randrange(1, 5))+' '+randomword(randrange(1, 5))+' '+randomword(randrange(1, 5))
        threads[n] = Thread(target=thread_task, args=([stringa,results,n])) 
        threads[n].start()
    for n in range(len(threads)):
        threads[n].join()
    for k in range(len(threads)):
        logger.info(results[k])
    process_name = current_process().name
    logger.info(f'| Process {process_name} done. |')
    
if __name__ == '__main__':
    queue = Queue()
    logger = logging.getLogger('app')
    logger.addHandler(QueueHandler(queue))
    logger.setLevel(logging.DEBUG)
    logger_p = Process(target=logger_process, args=(queue,), daemon=True)
    logger_p.start()
    logger.info('Main process started.')
    processes = [Process(target=process_task, args=(queue,)) for i in range(5)]
    for process in processes:
        process.start()
    for process in processes:
        process.join()
    logger.info('Main process done.')
    queue.put(None)
